In [1]:
import gpcrdb
import matplotlib
matplotlib.use('Agg')
matplotlib.rc('pdf', fonttype=42)
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = "Arial"
import ensembl
from utils import VariationType, Segment
import json
from scipy import stats
import pprint

In [11]:
num_high_frequent_missense = {s: 0 for s in Segment} # Fig. 2a

names_high_frequent_missense = []
annos_high_frequent_missense = []
is_A_high_frequent_missense = []
seg_length_with_high_frequent_missense = {s: 0 for s in Segment}

for receptor in gpcrdb.get_filtered_receptor_list():
    ensembl_entry = ensembl.EnsemblGeneEntry(receptor)
    with open(receptor.ensembl_path) as f:
        display_name = json.load(f)['display_name']
    with open(receptor.japan_cds_csv_path) as f:
        num_missense = 0

        for l in f.readlines():
            try:
                anno = ensembl.Annotation.from_csv_line(l)
                if anno.segment == Segment.FailedToGuess:
                    if anno.snv.rsid in ('rs376593544', 'rs1249120368'):
                        # Alignment-related issues.
                        # These variations are located between H8 and C-term in CCR2.
                        # (See `Class A (Rhodopsin)/ccr2_human/alignment.csv`)
                        # We reagard these variations as C-term for the following analysis.
                        anno.segment = Segment.Cterm
                    else:
                        raise NotImplementedError
                elif anno.segment == Segment.NONE:
                    raise NotImplementedError

                if anno.var_type == VariationType.MISSENSE and anno.snv.AF > 0.5:
                    num_missense += 1

                    num_high_frequent_missense[anno.segment] += 1

                    names_high_frequent_missense.append(display_name)
                    annos_high_frequent_missense.append(anno)
                    is_A_high_frequent_missense.append(receptor.receptor_class == 'Class A (Rhodopsin)')
            except ensembl.BlankLineError:
                continue
        if num_missense > 0:
            for s in Segment:
                seg_length_with_high_frequent_missense[s] += ensembl_entry.segments.count(s)

In [13]:
# Fig. 2a
fig, ax = plt.subplots(1, 1, figsize=(4, 1.5), dpi=300)
ax.set_facecolor('whitesmoke')

left = 0
for s in Segment:
    width = num_high_frequent_missense[s]
    if width == 0:
        continue

    ax.barh(0, width, height=0.7, left=left, color=s.color, edgecolor='k', lw=0.2)
    
    label = s.value
    x = left + width / 2
    if label.startswith('TM') or label.startswith('H8') or label.endswith('-term'):
        if label in ('TM1', 'N-term', 'H8', 'C-term'):
            ax.text(x, 0.375, label, ha='center', va='baseline', size=8)
        else:
            ax.text(x, 0.375, label[-1], ha='center', va='baseline', size=8)

    text = str(width) if s != Segment.Nterm else "{} variants".format(width)
    ax.text(x, -0.375, text, color='black', ha='center', va='top', size=8)

    left += width


ax.set_xlim(0, left)
ax.set_ylim(-0.5, 0.5)
ax.set_yticks([])
ax.set_yticklabels([])
ax.set_xlabel("Number of Missense Variants (AF > 0.5)")
fig.tight_layout()
fig.savefig("./figures/2a_high_allele_freq_vars.pdf")
plt.close(fig)


meta NOT subset; don't know how to subset; dropped


In [19]:
obs_N = num_high_frequent_missense[Segment.Nterm]
obs_non_N = sum(num_high_frequent_missense.values()) - obs_N

len_N = seg_length_with_high_frequent_missense[Segment.Nterm]
len_non_N = sum(seg_length_with_high_frequent_missense.values()) - len_N

M = len_N + len_non_N
n = len_N
N = obs_N + obs_non_N
k = obs_N

print(f"M = {M}, n = {n}, N = {N}, k = {k}")
p_hypergeom = stats.hypergeom.cdf(M=M, n=n, N=N, k=k)
print("N-term hypergemometric test p-value:", p_hypergeom)


M = 54358, n = 23093, N = 120, k = 42
N-term hypergemometric test p-value: 0.057480383009668234


In [16]:
# Fig. 2b
fig, ax = plt.subplots(1, 1, figsize=(4, 12), dpi=300)
ax.set_facecolor('whitesmoke')

sorted = [(name, anno, is_A) for name, anno, is_A in zip(names_high_frequent_missense, annos_high_frequent_missense, is_A_high_frequent_missense)]
sorted.sort(key=lambda arr: (arr[1].snv.AF, -arr[1].segment.index, arr[2], arr[1].residue_number))

A_ticks, A_ticklabels = [], []
non_A_ticks, non_A_ticklabels = [], []
for y, d in enumerate(sorted):
    name, anno, is_A = d[0], d[1], d[2]

    ax.barh(y, anno.snv.AF, color=anno.segment.color)

    if anno.snv.AF == 1:
        text = "1"
    else:
        text = "> {:.2f}".format(int(anno.snv.AF * 100) / 100)
    ax.text(anno.snv.AF + 0.01, y, text, size=5, ha='left', va='center')

    label = "{} {}{}{} ({})".format(name, anno.ref_aa, anno.residue_number, anno.alt_aa, 
                                    anno.generic_number if anno.generic_number else anno.segment.value)
    if is_A:
        A_ticks.append(y)
        A_ticklabels.append(label)
    else:
        non_A_ticks.append(y)
        non_A_ticklabels.append(label)

ax.set_yticks(A_ticks)
ax.set_yticklabels(A_ticklabels, size=7)
ax.set_yticks(non_A_ticks, minor=True)
ax.set_yticklabels(non_A_ticklabels, size=7, minor=True, color='tab:gray')
ax.set_xlabel("Allele Freq.")
ax.set_ylim(-0.8, len(sorted) - 0.3)
ax.set_xticks([0.5, 0.6, 0.8, 1.0])
ax.set_xticklabels([0.5, 0.6, 0.8, 1.0])
ax.set_xlim(left=0.5, right=1.1)
fig.tight_layout()
fig.savefig("./figures/2b_high_allele_freq_vars.pdf")

meta NOT subset; don't know how to subset; dropped


In [20]:
def _is_N_glycosylation_motif(triplet):
    assert(len(triplet) == 3)

    # Check if the variation is related with [N]-X-S/T motif, where X is not Proline.
    if triplet[0] == 'N' and triplet[1] != 'P' and triplet[2] in 'ST':
        return True
    return False

In [21]:
n_glyco_gain, n_glyco_loss = [], []
o_glyco_gain, o_glyco_loss = [], []
phospho_gain, phospho_loss = [], []
for receptor in gpcrdb.get_filtered_receptor_list():
    with open(receptor.ensembl_path) as f:
        display_name = json.load(f)['display_name']
    with open(receptor.alignment_path) as f:
        seq = ''.join([l.split(',')[0][0] for l in f if not l.startswith('#')])
    with open(receptor.japan_cds_csv_path) as f:
        for l in f:
            try:
                anno = ensembl.Annotation.from_csv_line(l)  
            except ensembl.BlankLineError:
                continue
            
            if anno.var_type != VariationType.MISSENSE:
                continue
            if anno.snv.AF <= 0.5:
                continue
            assert(seq[anno.residue_number - 1] == anno.ref_aa)
            
            # Check glycosylation
            if anno.segment in (Segment.Nterm, Segment.ECL1, Segment.ECL2, Segment.ECL3):    
                # N-glycosylation
                if 0 <= anno.residue_number - 3 and anno.residue_number <= len(seq):
                    ref_leading_triplet = seq[anno.residue_number - 3:anno.residue_number]
                    alt_leading_triplet = ref_leading_triplet[0] + ref_leading_triplet[1]+ anno.alt_aa

                    if _is_N_glycosylation_motif(ref_leading_triplet) != _is_N_glycosylation_motif(alt_leading_triplet):
                        if _is_N_glycosylation_motif(ref_leading_triplet):
                            # Loss
                            n_glyco_loss.append({"display_name": display_name, "annotation": anno, 
                                                "ref_motif": ref_leading_triplet, "alt_motif": alt_leading_triplet})
                        else:
                            # Gain
                            n_glyco_gain.append({"display_name": display_name, "annotation": anno,
                                                "ref_motif": ref_leading_triplet, "alt_motif": alt_leading_triplet})

                if 0 <= anno.residue_number - 2 and anno.residue_number + 1 <= len(seq):
                    ref_midmost_triplet = seq[anno.residue_number - 2:anno.residue_number + 1]
                    alt_midmost_triplet = ref_midmost_triplet[0] + anno.alt_aa + ref_midmost_triplet[2]

                    if _is_N_glycosylation_motif(ref_midmost_triplet) != _is_N_glycosylation_motif(alt_midmost_triplet):
                        if _is_N_glycosylation_motif(ref_midmost_triplet):
                            # Loss
                            n_glyco_loss.append({"display_name": display_name, "annotation": anno,
                                                "ref_motif": ref_midmost_triplet, "alt_motif": alt_midmost_triplet})
                        else:
                            # Gain
                            n_glyco_gain.append({"display_name": display_name, "annotation": anno,
                                                "ref_motif": ref_midmost_triplet, "alt_motif": alt_midmost_triplet})

                if 0 <= anno.residue_number - 1 and anno.residue_number + 2 <= len(seq):
                    ref_tailing_triplet = seq[anno.residue_number - 1:anno.residue_number + 2]
                    alt_tailing_triplet = anno.alt_aa + ref_tailing_triplet[1] + ref_tailing_triplet[2]

                    if _is_N_glycosylation_motif(ref_tailing_triplet) != _is_N_glycosylation_motif(alt_tailing_triplet):
                        if _is_N_glycosylation_motif(ref_tailing_triplet):
                            # Loss
                            n_glyco_loss.append({"display_name": display_name, "annotation": anno,
                                                "ref_motif": ref_tailing_triplet, "alt_motif": alt_tailing_triplet})
                        else:
                            # Gain
                            n_glyco_gain.append({"display_name": display_name, "annotation": anno,
                                                "ref_motif": ref_tailing_triplet, "alt_motif": alt_tailing_triplet})
                # O-glycosylation
                if anno.ref_aa in 'ST' and anno.alt_aa not in 'ST':
                    # Loss
                    o_glyco_loss.append({"display_name": display_name, "annotation": anno})
                elif anno.ref_aa not in 'ST' and anno.alt_aa in 'ST':
                    # Gain
                    o_glyco_gain.append({"display_name": display_name, "annotation": anno})
            # Check phosphorylation
            if anno.segment in (Segment.Cterm, Segment.ICL1, Segment.ICL2, Segment.ICL3, Segment.ICL4):
                if anno.ref_aa in 'ST' and anno.alt_aa not in 'ST':
                    # Loss
                    phospho_loss.append({"display_name": display_name, "annotation": anno})
                elif anno.ref_aa not in 'ST' and anno.alt_aa in 'ST':
                    # Gain
                    phospho_gain.append({"display_name": display_name, "annotation": anno})
n_glyco_gain.sort(key=lambda d: d['annotation'].snv.AF)
n_glyco_loss.sort(key=lambda d: d['annotation'].snv.AF)
o_glyco_gain.sort(key=lambda d: d['annotation'].snv.AF)
o_glyco_loss.sort(key=lambda d: d['annotation'].snv.AF)
phospho_gain.sort(key=lambda d: d['annotation'].snv.AF)
phospho_loss.sort(key=lambda d: d['annotation'].snv.AF)

In [31]:
len_C = seg_length_with_high_frequent_missense[Segment.Cterm]
len_ICLs = sum([seg_length_with_high_frequent_missense[s] for s in Segment.ICLs()])

obs_C, obs_ICLs = 0, 0
for p in phospho_gain + phospho_loss:
    if p['annotation'].segment == Segment.Cterm:
        obs_C += 1
    if p['annotation'].segment in Segment.ICLs():
        obs_ICLs += 1

M = len_C + len_ICLs
n = len_C
N = obs_C + obs_ICLs
k = obs_C

print(f"M = {M}, n = {n}, N = {N}, k = {k}")
p_hypergeom = stats.hypergeom.cdf(M=M, n=n, N=N, k=k)
print("Phosphorylation hypergemometric test p-value:", p_hypergeom)

M = 9104, n = 7178, N = 7, k = 7
Phosphorylation hypergemometric test p-value: 1.0


In [22]:
fig, axes = plt.subplots(3, 1, figsize=(4, 6), dpi=300, sharex=True, sharey=True)
for ax in axes:
    ax.set_facecolor('whitesmoke')
    ax.set_yticks([0.5, 0.6, 0.7, 0.8, 0.9, 1])
    ax.set_yticklabels([0.5, 0.6, 0.7, 0.8, 0.9, 1])
    ax.set_ylabel("Allele Freq.")

ax = axes[0]
n_glyco = [[d['annotation'].snv.AF for d in n_glyco_gain], [d['annotation'].snv.AF for d in n_glyco_loss]]
ax.hist(n_glyco, bins=10, range=(0.5, 1), stacked=True, label=["Gain", "Loss"], orientation='horizontal',
        color=['tab:orange', 'tab:gray'])
ax.set_title("N-glycosylation (Asn-X-Ser/Thr)") 
for glyco in n_glyco_gain + n_glyco_loss:
    anno = glyco['annotation']
    from_to_label = "({}$\\rightarrow${})".format(glyco['ref_motif'], glyco['alt_motif'])
    superscript = anno.generic_number if anno.generic_number else anno.segment.value
    superscript = r'$^{\mathrm{' + superscript + r'}}$'
    label = "{} {}{}{}{} {}".format(glyco['display_name'], anno.ref_aa, anno.residue_number, superscript, anno.alt_aa, from_to_label)
    for r in range(50, 100, 5):
        bottom, top = r / 100, r / 100 + 0.05
        if bottom < anno.snv.AF <= top:
            ax.text(1, (bottom + top) / 2 - 0.005, " " + label, ha='left', va='center', size=9)

ax = axes[1]
o_glyco = [[d['annotation'].snv.AF for d in o_glyco_gain], [d['annotation'].snv.AF for d in o_glyco_loss]]
ax.hist(o_glyco, bins=10, range=(0.5, 1), stacked=True, label=["Gain", "Loss"], orientation='horizontal',
        color=['tab:orange', 'tab:gray'])
ax.set_title("O-glycosylation (Ser/Thr)")

ax = axes[2]
phospho = [[d['annotation'].snv.AF for d in phospho_gain], [d['annotation'].snv.AF for d in phospho_loss]]
ax.hist(phospho, bins=10, range=(0.5, 1), stacked=True, label=["Gain", "Loss"], orientation='horizontal',
        color=['tab:orange', 'tab:gray'])
ax.set_title("Phosphorylation (Ser/Thr)")
ax.legend(loc='lower right', ncol=2)
ax.set_xticks(range(6))
ax.set_xticklabels(range(6))
ax.set_xlabel("Number of Variants")

fig.tight_layout()    
fig.savefig("./figures/2cde_ptm.pdf")

meta NOT subset; don't know how to subset; dropped
